In [1]:
# Import necessary moduels
import os
from langchain_core.tools import tool
from langgraph.graph import MessagesState, START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
# !pip install finnhub-python
import finnhub
from display_graph import display_graph
import keyring

In [3]:
# Initiialize Finnhub API client
FINNHUB_API_KEY = keyring.get_password('finnhub', 'key_for_windows')
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

In [4]:
# Define the tool : querying stock prices using the Finnhub API
@tool
def get_stock_price(symbol: str):
    """Retrieve the latest stock price for the given symbol."""
    qoute = finnhub_client.quote(symbol)
    return f"The current price for {symbol} is ${qoute['c']}"

# Register the tool in the tool node
tools = [get_stock_price]
tool_node = ToolNode(tools)

In [5]:
# Set up the AI model
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
model = ChatOpenAI(model='gpt-4o-mini', api_key=OPENAI_API_KEY)
model = model.bind_tools(tools)

# Define the function that simulates reasoning and invokes the model
def agent_reasoning(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages":[response]}

# Define conditional logic to determine whether to continue or stop
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If theere are no tool calls, finish the process
    if not last_message.tool_calls:
        return "end"
    return "continue"       # Otherwise, continue to the next stop

In [6]:
# Build the React agent using LangGraph
workflow = StateGraph(MessagesState)

# Add nodes : agent reasoning and tool invocation (stock price retrieval)
workflow.add_node("agent_reasoning", agent_reasoning)
workflow.add_node("call_tool", tool_node)

# Define the flow
workflow.add_edge(START, "agent_reasoning")     # Start with reasoning

# Conditional edges: continue to tool call or end the process
workflow.add_conditional_edges(
    "agent_reasoning",
    should_continue,
    {
        "continue":"call_tool",     # Proceed to get stock price
        "end":END                   # End the workflow
    }
)

# Normal edge: after invoking the tool, return to agent reasoning
workflow.add_edge("call_tool", "agent_reasoning")

In [7]:
# set up memory for breakpoints and add a breakpoint before calling the tool
memory = MemorySaver()
app = workflow.compile(checkpointer=memory, interrupt_before=["call_tool"])

# Display the graph
display_graph(app)

d:\projects\github\learning\llm\langgraph\langgraph_blueprint\ch9/graphs/graph_15175.png


In [8]:
# Simulate user input for stock symbol
initial_input = {"messages":[
    {"role":"user", "content":"Should I buy AAPL stock today?"}
]}
thread = {"configurable":{"thread_id":"1"}}

In [9]:
# Run the agent reasoning step first
for event in app.stream(initial_input, thread, stream_mode="values"):
    print(event)

# Pausing for human approval before retrieving stock price
user_approval = input("Do you approve querying the stock price for AAPL? (yes/no)")

if user_approval.lower() == "yes":
    # Continue with tool invocation to get stock price
    for event in app.stream(None, thread, stream_mode="values"):
        print(event)
else:
    print("execution halted by user.")

{'messages': [HumanMessage(content='Should I buy AAPL stock today?', additional_kwargs={}, response_metadata={}, id='12505d27-fede-404e-81dd-1b960e077394')]}
{'messages': [HumanMessage(content='Should I buy AAPL stock today?', additional_kwargs={}, response_metadata={}, id='12505d27-fede-404e-81dd-1b960e077394'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_wojvHuzr4zp3Eqw0P1MNW3bR', 'function': {'arguments': '{"symbol":"AAPL"}', 'name': 'get_stock_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 56, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0376897a-09bc-4f52-88eb-8858b8